In [0]:
from urllib.request import urlopen
import pandas as pd
import time
import json
import matplotlib.pyplot as plt
import re

In [2]:
API_key = '65c5bd10-7e51-11ea-9587-5f3a0490d8a1'
icao_url = "https://v4p4sz5ijk.execute-api.us-east-1.amazonaws.com/anbdata/airports/covid19/restrictions?api_key={}&format=json".format(API_key)

json_data = urlopen(icao_url).read()

#Read json data
df = pd.DataFrame.from_dict(json.loads(json_data))

#drop rows if column value 'traffic' are considered missing
df = df.dropna(subset=['traffic']).reset_index(drop=True)
df

,cityName,airportCode,countryCode,longitude,notams,NoTraffic,airportName,countryName,Closed,latitude,traffic
0,Fujairah,OMFJ,ARE,56.32416666666667,"{""location"":{""739"":""OMFJ""},""StateCode"":{""739"":...",False,Fujairah Intl,UAE,False,25.112222222222222,"{""_id"":{""4417"":""OMFJ""},""departures"":{""4417"":1}..."
1,Maputo,FQMA,MOZ,32.57263888888889,NaN,False,Maputo,Mozambique,False,-25.920833333333334,"{""_id"":{""4558"":""FQMA""},""departures"":{""4558"":1}..."
2,Lucknow,VILK,IND,80.88333333333334,"{""location"":{""817"":""VILK"",""818"":""VILK""},""State...",False,Chaudhary Charan Singh,India,False,26.761944444444445,"{""_id"":{""3077"":""VILK""},""departures"":{""3077"":1}..."
3,Perth,YPPH,AUS,115.96694444444445,NaN,True,Perth Intl,Australia,False,-31.940277777777776,"{""_id"":{""467"":""YPPH""},""departures"":{""467"":18},..."
4,Maldonado,SULS,URY,-55.09805555555556,"{""location"":{""771"":""SULS""},""StateCode"":{""771"":...",False,Capitan Curbelo Intl,Uruguay,False,-34.85722222222223,"{""_id"":{""1570"":""SULS""},""departures"":{""1570"":2}..."
...,...,...,...,...,...,...,...,...,...,...,...
1035,Nairobi,HKJK,KEN,36.927775,NaN,True,Jomo Kenyatta Intl,Kenya,False,-1.3192388888888888,"{""_id"":{""550"":""HKJK""},""departures"":{""550"":14},..."
1036,Termez,UTST,UZB,67.30833333333334,NaN,False,Termez,Uzbekistan,False,37.28666666666666,"{""_id"":{""1009"":""UTST""},""departures"":{""1009"":4}..."
1037,Cardiff,EGFF,GBR,-3.3433333333333333,"{""location"":{""847"":""EGFF"",""848"":""EGFF""},""State...",True,Cardiff,UK,False,51.39666666666667,"{""_id"":{""92"":""EGFF""},""departures"":{""92"":96},""c..."
1038,West Palm Beach,KPBI,USA,-80.09558333333332,NaN,True,Palm Beach Intl,USA,False,26.68316666666667,"{""_id"":{""654"":""KPBI""},""departures"":{""654"":10},..."


In [0]:
#Flattening JSON objects in Python
#Credits: https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '.')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '.')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [0]:
df2 = pd.DataFrame(columns=['_id','departures','connections','all_departures','all_connections','all_countries',
                               'week_departure_diff_pc', 'week_connections_diff_pc', 'ref_departure_diff_pc', 'ref_connections_diff_pc', 'date', 'airport'])

for i in range(len(df)):
  d = df['traffic'][i]
  rows = json.loads(d)

  #flat = flatten_json(rows)
  flattened_traffic = pd.json_normalize(rows)
  flattened_traffic.columns = flattened_traffic.columns.str.replace(r'\..*',"")

  df2 = df2.append(flattened_traffic)
  df2

In [0]:
#Single Case Test

#d = df['traffic'][3]
#rows = json.loads(d)

#flat = flatten_json(rows)
#flattened_traffic = pd.json_normalize(flat)
#flattened_traffic.columns = flattened_traffic.columns.str.replace(r'\..*',"")
#flattened_traffic

In [6]:
#Place the DataFrames side by side
horizontal_stack = pd.merge(df, df2, left_on='airportCode', right_on='_id')

#Drop unused columns and renames columns to make them readable by others
df_final = horizontal_stack.drop(['notams','traffic', '_id','connections','all_departures','all_connections','all_countries','week_connections_diff_pc','ref_connections_diff_pc', 'airport'] , axis='columns')

df_final = df_final.rename(columns={"week_departure_diff_pc": "previous_week_dep_percent_change", "ref_departure_diff_pc": "reference_week_dep_percent_change"})

df_final

,cityName,airportCode,countryCode,longitude,NoTraffic,airportName,countryName,Closed,latitude,departures,previous_week_dep_percent_change,reference_week_dep_percent_change,date
0,Fujairah,OMFJ,ARE,56.32416666666667,False,Fujairah Intl,UAE,False,25.112222222222222,1,None,0.000000,2020-04-12
1,Maputo,FQMA,MOZ,32.57263888888889,False,Maputo,Mozambique,False,-25.920833333333334,1,-66.6667,-97.674419,2020-04-12
2,Lucknow,VILK,IND,80.88333333333334,False,Chaudhary Charan Singh,India,False,26.761944444444445,1,0,-97.826087,2020-04-12
3,Perth,YPPH,AUS,115.96694444444445,True,Perth Intl,Australia,False,-31.940277777777776,18,-33.3333,-91.133005,2020-04-12
4,Maldonado,SULS,URY,-55.09805555555556,False,Capitan Curbelo Intl,Uruguay,False,-34.85722222222223,2,None,0.000000,2020-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,Nairobi,HKJK,KEN,36.927775,True,Jomo Kenyatta Intl,Kenya,False,-1.3192388888888888,14,0,-95.409836,2020-04-12
1036,Termez,UTST,UZB,67.30833333333334,False,Termez,Uzbekistan,False,37.28666666666666,4,None,0.000000,2020-04-12
1037,Cardiff,EGFF,GBR,-3.3433333333333333,True,Cardiff,UK,False,51.39666666666667,96,None,0.000000,2020-04-12
1038,West Palm Beach,KPBI,USA,-80.09558333333332,True,Palm Beach Intl,USA,False,26.68316666666667,10,-9.09091,-76.744186,2020-04-12


In [0]:
import os
# define the name of the directory to be created
path = "/Airport_COVID19_NOTAMs"

if not os.path.exists(path):
    os.makedirs(path)

# To write dataframe to new Excel file and append data to 'Aggregated_Fortress_headsets_offer.csv'
CurrentDate = time.strftime("%Y-%m-%d")

df_final.to_excel(path + '/' + CurrentDate + '_ICAO_Airport_COVID19_NOTAMs' + '.xlsx', sheet_name = 'Airport_COVID19_dep', index = False)

#Stack the DataFrames on top of 'ICAO_Airport_COVID-19_NOTAMs.csv'
df_final.to_csv(path + '/' + 'latest_ICAO_Airport_COVID19_NOTAMs.csv', index = False, header = False, mode='a')